<a href="https://colab.research.google.com/github/ignaciomontovio/TP-Virus/blob/Nacho/TP_EntregableVirus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Examen Práctico

#### 01-3900 | Ciencia de datos | 2024

Alumno:

## Enunciado

Se tienen un dataset con datos de pacientes internados en un hospital (TP_Virus_Alumnos.csv). La clase de interes (1) refiere a la presencia de un virus. El virus tiene normalmente una gravedad leve/baja y el tratamiento suele ser invasivo. Datos como nombre y apellido han sido eliminados y los valores tanto en sangre (BLD), hormonales u otros análisis sobre reactivos han sido alterados en sus valores para preservar la privacidad. Se aclara que no se ha modificado su capacidad predictiva (Si es que la tienen).


Para su conocimiento: </BR>
Datos generales de Edad, Peso, Altura y condición laboral (Activo, Pasivo etc).
Datos medidos en hospital:</BR>
BLD: Sangre</BR>
LVL: Hormonales</BR>
REC: Otros análisis</BR>

Se pide obtener con los datos disponibles el mejor modelo posible que prediga la presencia o ausencia del virus.
Dado que el tratamiento es invasivo y la grevedad es moderada se requiere "atrapar" tantos "1" como sea posible y minimizar los falsos positivos para evitar que reciban un tratamiento de estas caracteristicas personas que no presentan el virus. Intente obtener el mejor modelo que maximice la métrica que considere correspondiente.



## Como desarrollar el exámen

A partir del dataset realice todas las acciones para poder llegar al mejor modelo, explique brevemente en los fundamentos de sus transformaciones o acciones en general.

La nota derivará de: </BR>
1.La calidad de la clasificación realizada</BR>
2.La fundamentación de los pasos realizados</BR>
3.Lo sencillo de llevar a producción el desarrollo</BR>



Los docentes evaluaran su clasificador utilizando un conjunto de datos del dataset "fuera de la caja" (out of the box, al que usted no tiene acceso). Para minimizar la posible diferencia entre su medición y la medición del docente recuerde y aplique conceptos de test, validación cruzada y evite los errores comunes de sesgo de selección y fuga de datos (PPT/Pdf Árboles de clasificación) o  Sklearn "10. Common pitfalls and recommended practices" disponible en "https://scikit-learn.org/stable/common_pitfalls.html"   

Al final del notebook encontrará un bloque de código que lee la muestra adicional (a la que usted no tiene acceso) si PRODUCCION==True, en caso contrario solo lee una submuestra del conjunto original para validar que el código funciona. Desarrolle el notebook como considere, para finalmente asignar el mejor clasificador que usted haya obtenido remplazando en f_clf = None, None por su clasificador. Implemente todas las transformaciones entre esa línea y la predición final (Evitando al fuga de datos). Ver TP_AutomatizarTransformaciones.ipynb

En materiales del MIEL se adjunta un notebook que propone algunas ideas para automatizar el proceso.

## Evaluacion final - Docente + Alumno

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, OrdinalEncoder


PRODUCCION = False
best_clf = None #Asignar aqui el mejor clasificador posible (previamente entrenado)

#Leemos el dataset de evaluación, simulando producción
if PRODUCCION==False:
    df = pd.read_csv("TP_Virus_Alumnos.csv")
    # df_orig = df
    # _, df = train_test_split(df, test_size=0.3, random_state=42)
else:
    df = pd.read_csv("TP_Virus_Evaluacion.csv")
#Dividimos en target y predictoras

X_prod = df.drop("target", axis=1)
y_prod = df["target"]

#Transformaciones


#Evaluación final
#y_pred = best_clf.predict(X_prod)
#print(classification_report(y_prod, y_pred))
# df.shape[0]

## Analisis de Variables

In [ ]:
# verificamos los tipos de datos
df.dtypes

In [ ]:
# Verificamos si hay valores nulos para imputar
print(df.isnull().sum())
# Verificamos si hay duplicados
print(df.duplicated().sum())

### Correlacion

In [ ]:
#En la matriz de correlación vemos que edad, peso e hijos estan ampliamente correlacionados.
#Las demas variables no poseen correlacion entre ellas, pueden aportar valor.
fig, ax = plt.subplots(figsize=(10,7))
mat = df.select_dtypes(include=['int', 'float']).corr()
sns.heatmap(mat, annot=True, cmap='coolwarm', fmt=".2f",  ax=ax)
plt.title('Matriz de Correlación')
plt.show()

### Edad

In [ ]:
# Analisis de la distribución de la variable target "Edad"
print( df.Edad.value_counts() )
sns.countplot(x='Edad', data=df, hue='target', legend=False)

In [ ]:
df[["Edad"]].boxplot()
df[["Edad"]].hist()

In [ ]:
df[["Edad"]].describe().T

In [ ]:
df.Edad.value_counts()

### Peso

In [ ]:
df[["Peso"]].boxplot()
df[["Peso"]].hist()

In [ ]:
df[["Peso"]].describe().T

In [ ]:
df.Peso.value_counts()

# Observamos outliers en peso. Los valores atipicos estan relacionados con los recien nacidos, debido a la media del peso de la muestra,
# parece ser un valor fuera de lo normal cuando no lo es. Sin embargo que el peso '8.934178..'
# sea el único que encuentre repetido varias veces es indicativo de un error en la carga o defectos en la muestra.

### Hijos

In [ ]:
# Analisis de la distribución de la variable target "hijos"
print( df.hijos.value_counts() )
sns.countplot(x='hijos', data=df, hue='hijos', legend=False)

df[["hijos"]].boxplot()
df[["hijos"]].hist()

In [ ]:
df[["hijos"]].describe().T

In [ ]:
df.hijos.value_counts()

### BLD01

In [ ]:
df[["BLD01"]].boxplot()
df[["BLD01"]].hist(bins=30)

In [ ]:
df[["BLD01"]].describe().T

In [ ]:
df.BLD01.value_counts()

### BLD02

In [ ]:
df[["BLD02"]].boxplot()
df[["BLD02"]].hist()

In [ ]:
df[["BLD02"]].describe().T

In [ ]:
print(df.BLD02.value_counts())

### BLD03

In [ ]:
df[["BLD03"]].boxplot()
df[["BLD03"]].hist()

In [ ]:
df[["BLD03"]].describe().T

In [ ]:
df.BLD03.value_counts()

### REC1

In [ ]:
df[["REC1"]].boxplot()
df[["REC1"]].hist()

In [ ]:
df[["REC1"]].describe().T

In [ ]:
df.REC1.value_counts()

### REC2

In [ ]:
df[["REC2"]].boxplot()
df[["REC2"]].hist()

In [ ]:
df[["REC2"]].describe().T

In [ ]:
df.REC2.value_counts()

### REC3

In [ ]:
df[["REC3"]].boxplot()
df[["REC3"]].hist()

In [ ]:
df[["REC3"]].describe().T

In [ ]:
df.REC3.value_counts()

### REC4

In [ ]:
df[["REC4"]].boxplot()
df[["REC4"]].hist()

In [ ]:
df[["REC4"]].describe().T

In [ ]:
df.REC4.value_counts()

### REC5

In [ ]:
df[["REC5"]].boxplot()
df[["REC5"]].hist()

In [ ]:
df[["REC5"]].describe().T

In [ ]:
df.REC5.value_counts()

### Target

In [ ]:
sns.countplot(x='target', data=df, hue='target', legend=False)

### Conclusiones y preguntas

Las variables BLD01, BLD02, BLD03, REC1, REC2, REC3, REC4 y REC5, tienen outliers casi en ambos extremos, dice en el enunciado que estas variables fueron alteradas en sus valores para preservar la privacidad de las personas. Quizas eso afecto su distribución?
Estoy dudando si eliminarlas o tratarlas porque las variables Peso y LVL tambien muestran outliers pero en sus limites inferiores. Que se explica con que la muestra contiene recien nacidos y personas muy jovenes, por lo que no creo que se necesite tratarlas.

In [ ]:
#No hay duplicados.
print("Cantidad:",  df.duplicated().sum())

## Tratamiento de variables

Aca vamos a tratar cosas generales que queremos aplicar siempre, luego en la evaluacion del modelo,
con los pipelines probamos y analiamos distintas estragias de imputacion, normalizacion, etc...

In [ ]:
#Elimino los nulos en LVL
df_normalize = df.dropna(subset=['LVL'])
df_normalize = df_normalize.dropna(subset=['Edad'])

In [ ]:
def preparacion_de_datos(df_input, target, escalar_valores=False):
  df = df_input
  if escalar_valores:
    scaler_X = StandardScaler(with_mean=True, with_std=True)
    scaler_X.fit(df.drop(target,axis=1))
    x = pd.DataFrame(scaler_X.transform(df.drop(target,axis=1),), columns = df.drop(target,axis=1).columns )
  else:
    x = df.drop(target,axis=1)
  y = df[target]
  return x,y

In [ ]:
X_virus, y_virus = preparacion_de_datos( df_normalize, "target" ,escalar_valores=False)

In [ ]:
# Hacemos el Split 70-30 para train-test
X_train, X_test, y_train, y_test = train_test_split(X_virus, y_virus,
                                                    test_size=0.3, stratify = y_virus, random_state=0)

## Utilidades - Imputacion

In [ ]:
def graficarCurvaRoc( y_pred, model ):
  fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred)
  auc = metrics.roc_auc_score(y_test, y_pred)
  # Graficamos
  plt.plot(fpr,tpr,label= model +" AUC="+str(round(auc,4))) #,label= "AUC="+str(auc))
  plt.legend(loc=4, fontsize=12)
  return auc

In [ ]:
# Para automatizar la imputacion y hacerlo mas rapido y simple
# Nos va a servir para probar rapido distintas strategias de imputacion, encoding, normalizacion
# Y ver cual es la mejor.

class ColImputer(BaseEstimator, TransformerMixin):
    def __init__(self, imputer=SimpleImputer(), columns=[]):
        super().__init__()
        self.imputer = imputer
        self.columns = columns

    def fit(self, X, y=None):
        self.imputer.fit(X[self.columns])
        return self

    def get_feature_names_out(self):
        return self.imputer.get_feature_names_out()

    def  transform(self, X):
        Xc = X.copy()
        Xc.loc[:, self.columns] = self.imputer.transform(Xc[self.columns])
        return Xc

class ColEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, encoder=None, columns=[]):
        super().__init__()
        self.encoder = encoder
        self.columns = columns

    def fit(self, X, y=None):
        self.encoder.fit(X[self.columns])
        return self

    def get_feature_names_out(self):
        return self.get_feature_names_out()

    def  transform(self, X):
        Xc = X.copy()
        Xc.loc[:, self.columns] = self.encoder.transform(Xc[self.columns])
        return Xc

class ColScaler(BaseEstimator, TransformerMixin):
    def __init__(self, scaler=StandardScaler(), columns=[]):
        super().__init__()
        self.scaler = scaler
        self.columns = columns

    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns])
        return self

    def get_feature_names_out(self):
        return self.scaler.get_feature_names_out()

    def  transform(self, X):
        Xc = X.copy()
        Xc.loc[:, self.columns] = self.scaler.transform(Xc[self.columns])
        return Xc

class ColDummy(BaseEstimator, TransformerMixin):
    def __init__(self, columns=[], delete = ''):
        super().__init__()
        self.columns = columns
        self.delete = delete

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        Xc = X.copy()
        Xc = pd.get_dummies(Xc, columns=self.columns)
        if(self.delete != ''):
          Xc = Xc.drop(columns=self.delete)
        #Xc = X.drop(columns=self.columns)
        return Xc

class ReplaceValue(BaseEstimator, TransformerMixin):
    def __init__(self, column, old_value, new_value):
        self.column = column
        self.old_value = old_value
        self.new_value = new_value

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        Xc = X.copy()
        Xc.loc[:, self.column] = Xc[self.column].replace(self.old_value, self.new_value)
        return Xc

## Evaluacion de modelos

In [ ]:
df_norm = df
df_norm.loc[df['LVL'] == 1000000, 'LVL'] = None

#df_norm = df_norm[df_norm['Edad'] > 10]
#df_norm = df_norm[df_norm['Peso'] != 8.934178194834107]

#df_norm = df_norm.drop(columns=['edad'])

# Reemplazar los valores NaN por 'Desconocido' en Genero
df_norm["Genero"].fillna('Desconocido', inplace=True)


# Reemplazar los valores NaN por 'Desconocido' en Edad
#df_norm["Edad"].fillna('Desconocido', inplace=True)

X = df_norm.drop("target", axis=1)
y = df_norm["target"]

X_train_norm, X_test_norm, y_train, y_test = train_test_split(X, y, random_state=3, test_size=0.3)


pl = Pipeline(steps=[
    ("LVLImputer", ColImputer(imputer=SimpleImputer(strategy='mean'), columns=["LVL"])),
    ("EdadImputer", ColImputer(imputer=SimpleImputer(strategy='mean'), columns=["Edad"])),
    ("LaboralDummies", ColDummy(columns=["Laboral"])),
    ("GeneroDummies", ColDummy(columns=["Genero"], delete='Genero_Desconocido')),
    ("BLD03Scaler", ColScaler(scaler=MinMaxScaler(), columns=["BLD03"])),
    ("BLD02Scaler", ColScaler(scaler=MinMaxScaler(), columns=["BLD02"])),
    ("BLD01Scaler", ColScaler(scaler=MinMaxScaler(), columns=["BLD01"])),
    ("LVLScaler", ColScaler(scaler=MinMaxScaler(), columns=["LVL"]))
])

pl.fit(X_train_norm, y_train)
X_train = pl.transform(X_train_norm)
X_test = pl.transform(X_test_norm)

X_train

### Armado de modelos y obtencion de precision

In [ ]:
logreg = LogisticRegression( max_iter=3000 )
logreg.fit(X_train, y_train)
y_pred_lg = logreg.predict(X_test)

treeclf = DecisionTreeClassifier(max_depth=10, random_state=1)
treeclf.fit(X_train, y_train)
y_pred_tc = treeclf.predict(X_test)

bayes_multi = MultinomialNB()
bayes_multi.fit(X_train, y_train)
y_pred_nb = bayes_multi.predict(X_test)

bayes_gauss = GaussianNB()
bayes_gauss.fit(X_train, y_train)
y_pred_gauss = bayes_gauss.predict(X_test)

In [ ]:
# Inicializamos los labels del gráfico
plt.figure(figsize=(20, 10))
plt.xlabel('% 1 – Specificity (falsos positivos)', fontsize=14)
plt.ylabel('% Sensitivity (positivos)', fontsize=14)

# Graficamos la recta del azar
it = [i/100 for i in range(100)]
plt.plot(it,it,label="AZAR AUC=0.5",color="black")

modelos = { 'bayesGauss':y_pred_gauss,'arbol':y_pred_tc, 'reglog':y_pred_lg, 'multinomial': y_pred_nb}
areas = []
for pred in modelos:
    auc = graficarCurvaRoc( modelos[pred] , pred )
    areas.append( (pred, auc) )
areas = pd.DataFrame(areas, columns=['model','auc'])
# Grafico
# plt.title("Curva ROC", fontsize=14)
# plt.tick_params(labelsize=12);
# plt.show()
# Tabla
areas.sort_values('auc', ascending=False)

### Clasificacion de todos los modelos.
Debemos mejorar precision: para evitar los falsos positivos, dado que el tratamiento es invasivo y la gravedad moderada.
Debemos mejorar recall: porque queremos detectar todos los positivos posibles.

In [ ]:
from IPython.display import display, Markdown

display(Markdown(f"## LogisticRegression"))
print(classification_report(y_test, y_pred_lg))

display(Markdown(f"## DecisionTreeClassifier"))
print(classification_report(y_test, y_pred_tc))

display(Markdown(f"## MultinomialNB"))
print(classification_report(y_test, y_pred_nb))

display(Markdown(f"## GaussianNB"))
print(classification_report(y_test, y_pred_gauss))


## Mejora de los modelos con GridSearch

### Logistic Regression

In [ ]:
from warnings import simplefilter
# ignore all warnings
simplefilter(action='ignore')

# Con todas las opciones de solver no converge nunca
# "solver": ['saga', 'sag', 'newton-cholesky', 'newton-cg', 'liblinear', 'lbfgs']

parameters =  {"C":np.logspace(-3,3,13), "penalty":["l1","l2",None], "max_iter":[1500,3000,4000]}
clf = GridSearchCV( LogisticRegression() , parameters, scoring='precision', cv=5, )
clf.fit(X_train, y_train)

print("Best Parameters:", clf.best_params_)
print("Best Score:", clf.best_score_)

### KNN

In [ ]:
parameters = {"criterion": ['gini', 'entropy', 'log_loss'], "splitter": ['best', 'random'], "max_depth": [2, 5, 10, 20, 30, 40], "random_state": [1,5,9,15,20,30,40]}
clf = GridSearchCV( DecisionTreeClassifier() , parameters, scoring='precision', cv=5, )
clf.fit(X_train, y_train)

print("Best Parameters:", clf.best_params_)
print("Best Score:", clf.best_score_)

### Multinomial

In [ ]:
parameters = {"fit_prior": [True, False], "alpha": [0,1.0,2.0,3.0], "force_alpha": [True, False]}
clf = GridSearchCV( MultinomialNB() , parameters, scoring='precision', cv=5, )
clf.fit(X_train, y_train)

print("Best Parameters:", clf.best_params_)
print("Best Score:", clf.best_score_)

### GaussianNB

In [ ]:
parameters = {'var_smoothing': np.logspace(0,-9, num=100)}
clf = GridSearchCV( GaussianNB() , parameters, scoring='precision', cv=5, )
clf.fit(X_train, y_train)

print("Best Parameters:", clf.best_params_)
print("Best Score:", clf.best_score_)

## Re-Evaluacion

In [ ]:
logreg = LogisticRegression(max_iter=1500, penalty='l2', C=np.float64(0.31622776601683794))
logreg.fit(X_train, y_train)
y_pred_lg = logreg.predict(X_test)


treeclf = DecisionTreeClassifier(max_depth=5, random_state=20, splitter='random', criterion='entropy')
treeclf.fit(X_train, y_train)
y_pred_tc = treeclf.predict(X_test)

bayes_multi = MultinomialNB(alpha=2.0, fit_prior=True, force_alpha=True)
bayes_multi.fit(X_train, y_train)
y_pred_nb = bayes_multi.predict(X_test)

bayes_gauss = GaussianNB(var_smoothing=np.float64(1.0))
bayes_gauss.fit(X_train, y_train)
y_pred_gauss = bayes_gauss.predict(X_test)

plt.figure(figsize=(20, 10))
plt.xlabel('% 1 – Specificity (falsos positivos)', fontsize=14)
plt.ylabel('% Sensitivity (positivos)', fontsize=14)

# Graficamos la recta del azar
it = [i/100 for i in range(100)]
plt.plot(it,it,label="AZAR AUC=0.5",color="black")

modelos = { 'bayesGauss':y_pred_gauss,'arbol':y_pred_tc, 'reglog':y_pred_lg, 'multinomial': y_pred_nb}
areas = []
for pred in modelos:
    auc = graficarCurvaRoc( modelos[pred] , pred )
    areas.append( (pred, auc) )
areas = pd.DataFrame(areas, columns=['model','auc'])
# Grafico
# plt.title("Curva ROC", fontsize=14)
# plt.tick_params(labelsize=12);
# plt.show()
# Tabla
areas.sort_values('auc', ascending=False)

In [ ]:
from IPython.display import display, Markdown

display(Markdown(f"## LogisticRegression"))
print(classification_report(y_test, y_pred_lg))

display(Markdown(f"## DecisionTreeClassifier"))
print(classification_report(y_test, y_pred_tc))

display(Markdown(f"## MultinomialNB"))
print(classification_report(y_test, y_pred_nb))

display(Markdown(f"## GaussianNB"))
print(classification_report(y_test, y_pred_gauss))

## Validación

Para garantizar la robustez y la generalización de nuestro modelo utilizaremos la técnica de validación cruzada (K-Folds). Esto nos permitirá evaluar el rendimiento del modelo de manera más exhaustiva.

La validación cruzada nos ayudará a reducir el sesgo de evaluación,Además nos permitirá identificar la variabilidad en el rendimiento del modelo, minimizando así el riesgo de sobreajuste (overfitting) y asegurando que el modelo generalice bien a nuevos datos no vistos.

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

def realizar_kfolds_val(model,X,y,splits=5,shf=True):
    kf = KFold(n_splits=splits, shuffle=shf, random_state=42)

    # Evaluar el modelo utilizando validación cruzada
    scores = cross_val_score(model, X, y, cv=kf)

    print(f'Scores for each fold: {scores}')
    print(f'Mean score: {scores.mean()}')
    print(f'Standard deviation: {scores.std()}')

print("Cross-Validation - Log Reg")
realizar_kfolds_val(logreg,X_train,y_train,5)

print("\n\nCross-Validation - Decision Tree Classifier")
realizar_kfolds_val(treeclf,X_train,y_train,5)

print("\n\nCross-Validation - Multinomial Bayes")
realizar_kfolds_val(bayes_multi,X_train,y_train,5)

print("\n\nCross-Validation - Gaussian Bayes")
realizar_kfolds_val(bayes_gauss,X_train,y_train,5)

## Pendientes:

Ver si podemos balancer la variable target, (todas las metricas nos dab bajas para el caso de positivos)
Tratar las variables faltantes solo estamos haciendo:
("LVLImputer", ColImputer(imputer=SimpleImputer(strategy='mean'), columns=["LVL"])),
    ("EdadImputer", ColImputer(imputer=SimpleImputer(strategy='mean'), columns=["Edad"])),
    ("LaboralDummies", ColDummy(columns=["Laboral"])),
    ("GeneroDummies", ColDummy(columns=["Genero"], delete='Genero_Desconocido')),
    ("BLD03Scaler", ColScaler(scaler=MinMaxScaler(), columns=["BLD03"])),
    ("BLD02Scaler", ColScaler(scaler=MinMaxScaler(), columns=["BLD02"])),
    ("BLD01Scaler", ColScaler(scaler=MinMaxScaler(), columns=["BLD01"])),
    ("LVLScaler", ColScaler(scaler=MinMaxScaler(), columns=["LVL"]))
Probar otros scalers, strategias etc...
Ver validacion Cruzada
Ver sesgo de seleccion y fuga de datos

en la parte de tratamiento de variables no estamos haciendo nada relevante, hace falta ese scaler a todo el dataset? o podemos tener los scalers del pipeline y ya?
por lo que veo en los colab escala casi siempre todas las variables creo deberiamos hacerlo

en varios colab elige que variables utilizar y cuales no (en base a la correlacion?) ver si eso mejora las metricas